Example of building a simple CNN for CIFAR10 dataset. 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision.models as models

In [3]:
dataset_dir = 'dataset_preprocessed'

In [4]:
# #calculate means and std to be used with normalization
# dataset = ImageFolder(dataset_dir, transform=ToTensor())

# # Compute mean and standard deviation
# means = []
# stds = []
# for img, _ in dataset:
#     means.append(img.mean(dim=(1,2)))
#     stds.append(img.std(dim=(1,2)))

# means = np.stack(means).mean(axis=0)
# stds = np.stack(stds).mean(axis=0)

# print(f"Means: {means}")
# print(f"Stds: {stds}")

means= [0.3337701, 0.35129565, 0.36801142]
stds= [0.16881385, 0.1562263, 0.16852096]


In [6]:
#Load resnet50 and freeze
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

# Replace the last layer for 2 classes only
features = resnet50.fc.in_features
resnet50.fc = nn.Linear(features, 2)

c:\Users\ammar\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ammar\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ammar/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:28<00:00, 3.54MB/s]


In [7]:
#hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001


"""
Creates a data transformation pipeline using the transforms.Compose function from the PyTorch library.

"""
transform = transforms.Compose(
    [
     transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize((means[0],means[1],means[2]), (stds[0],stds[1],stds[2]))])

# Load the training and validation datasets in folder 'dataset'
full_data = ImageFolder(dataset_dir, transform=transform)

#Split the dataset
train_size = int(0.85 * len(full_data))
test_size = len(full_data) - train_size
trainDataset, testDataset = torch.utils.data.random_split(full_data, [train_size, test_size])

# Create DataLoader objects to iterate over the datasets in batches
trainloader = torch.utils.data.DataLoader(trainDataset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testDataset, batch_size=32)


In [8]:
net = resnet50
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Train the network
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 5 == 4:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_acc = 100.0 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Train Acc: {train_acc:.2f}%')

print('Finished Training')

[1,     5] loss: 0.645
[1,    10] loss: 0.554
[1,    15] loss: 0.478
[1,    20] loss: 0.381
[1,    25] loss: 0.335
Epoch 1/10, Train Acc: 77.01%
[2,     5] loss: 0.310
[2,    10] loss: 0.259
[2,    15] loss: 0.261
[2,    20] loss: 0.312
[2,    25] loss: 0.244
Epoch 2/10, Train Acc: 91.95%
[3,     5] loss: 0.209
[3,    10] loss: 0.169
[3,    15] loss: 0.262
[3,    20] loss: 0.227
[3,    25] loss: 0.240
Epoch 3/10, Train Acc: 93.10%
[4,     5] loss: 0.163
[4,    10] loss: 0.141
[4,    15] loss: 0.171
[4,    20] loss: 0.191
[4,    25] loss: 0.166
Epoch 4/10, Train Acc: 95.02%
[5,     5] loss: 0.242
[5,    10] loss: 0.179
[5,    15] loss: 0.202
[5,    20] loss: 0.127
[5,    25] loss: 0.141
Epoch 5/10, Train Acc: 94.25%
[6,     5] loss: 0.154
[6,    10] loss: 0.123
[6,    15] loss: 0.188
[6,    20] loss: 0.141
[6,    25] loss: 0.147
Epoch 6/10, Train Acc: 95.66%
[7,     5] loss: 0.122
[7,    10] loss: 0.155
[7,    15] loss: 0.082
[7,    20] loss: 0.124
[7,    25] loss: 0.147
Epoch 7/10, Tra

In [9]:
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [10]:
print('Accuracy of the network on the '+str(test_size)+' test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 139 test images: 94 %


In [11]:

# Save the trained model
PATH = './floodFinal.pth'
torch.save(net.state_dict(), PATH)

In [12]:
#Testing on random images from internet

# model = resnet50
# model.load_state_dict(torch.load('floodFinal.pth'))
# model.eval()


# transform = transforms.Compose(
#     [
#      transforms.Resize((256,256)),
#      transforms.ToTensor(),
#      transforms.Normalize((means[0],means[1],means[2]), (stds[0],stds[1],stds[2]))])

# full_data = ImageFolder('data', transform=transform)

# testloader = torch.utils.data.DataLoader(full_data, batch_size=32)

# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         print(labels)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(correct)
# print(total)

# print('Accuracy of the network on the '+str(test_size)+' test images: %d %%' % (
#     100 * correct / total))

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 1])
9
10
Accuracy of the network on the 139 test images: 90 %
